# Regression
***

# Import

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from vecstack import stacking

# Prepare data

In [2]:
boston = load_boston()
X, y = boston.data, boston.target

# Make train/test split
# As usual in machine learning task we have X_train, y_train, and X_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize 1st level models

In [3]:
# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended.
models = [
    ExtraTreesRegressor(random_state=0, n_jobs=-1, 
                        n_estimators=100, max_depth=3),
        
    RandomForestRegressor(random_state=0, n_jobs=-1, 
                          n_estimators=100, max_depth=3),
        
    XGBRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, 
                 n_estimators=100, max_depth=3)
]

# Perform stacking

In [4]:
S_train, S_test = stacking(models,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=True,            # regression task (if you need 
                                                       #     classification - set to False)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and find mean
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=mean_absolute_error, # metric: callable
                           n_folds=4,                  # number of folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)                  # print all info

task:       [regression]
metric:     [mean_absolute_error]
mode:       [oof_pred_bag]
n_models:   [3]

model 0:    [ExtraTreesRegressor]
    fold 0: [3.20733439]
    fold 1: [2.87943130]
    fold 2: [2.53026486]
    fold 3: [2.83618694]
    ----
    MEAN:   [2.86330437] + [0.23993093]
    FULL:   [2.86330437]

model 1:    [RandomForestRegressor]
    fold 0: [3.11110485]
    fold 1: [2.78404210]
    fold 2: [2.55707729]
    fold 3: [2.32209992]
    ----
    MEAN:   [2.69358104] + [0.29117900]
    FULL:   [2.69358104]

model 2:    [XGBRegressor]
    fold 0: [2.40318942]
    fold 1: [2.37286943]
    fold 2: [1.89121526]
    fold 3: [1.95382805]
    ----
    MEAN:   [2.15527554] + [0.23404984]
    FULL:   [2.15527554]



# Look at the result

So now we have OOF from 1st level models and we can build 2nd level model.  
But first let's look at the result.  
We have three 1st level models, so we expect to get three columns in `S_train` and `S_test`.  

In [5]:
S_train[:5]

array([[ 27.21782522,  28.23561508,  27.78520966],
       [ 22.25443115,  22.32927929,  22.57203102],
       [ 26.03879794,  25.80114661,  26.27923012],
       [ 21.82927308,  21.30478775,  21.39201546],
       [ 13.02143285,  12.04667683,   8.88440514]])

In [6]:
S_test[:5]

array([[ 24.89602382,  23.85490698,  24.85046005],
       [ 20.85135955,  25.05068336,  26.30952454],
       [ 23.13164045,  21.56864103,  23.67526102],
       [ 13.47709586,  11.81606315,  11.02050447],
       [ 21.93179664,  21.30652111,  21.75125122]])

# Apply 2nd level model

In [7]:
# Initialize 2nd level model
model = XGBRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, 
                     n_estimators=100, max_depth=3)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % mean_absolute_error(y_test, y_pred))

Final prediction score: [2.78409081]
